In [1]:
import pyspark
import xml.etree.cElementTree as ET
from os import listdir
from os.path import isfile, join

In [2]:
mypath = './states'
osmfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
osmfiles

['./states/missouri-latest.osm',
 './states/louisiana-latest.osm',
 './states/new-york-latest.osm',
 './states/new-hampshire-latest.osm',
 './states/west-virginia-latest.osm',
 './states/delaware-latest.osm',
 './states/new-jersey-latest.osm',
 './states/wyoming-latest.osm',
 './states/wisconsin-latest.osm',
 './states/washington-latest.osm',
 './states/north-carolina-latest.osm',
 './states/arizona-latest.osm',
 './states/nevada-latest.osm',
 './states/virginia-latest.osm',
 './states/oregon-latest.osm',
 './states/kentucky-latest.osm',
 './states/connecticut-latest.osm',
 './states/utah-latest.osm',
 './states/hawaii-latest.osm',
 './states/maryland-latest.osm',
 './states/ohio-latest.osm',
 './states/rhode-island-latest.osm',
 './states/iowa-latest.osm',
 './states/south-dakota-latest.osm',
 './states/maine-latest.osm',
 './states/nebraska-latest.osm',
 './states/illinois-latest.osm',
 './states/kansas-latest.osm',
 './states/mississippi-latest.osm',
 './states/north-dakota-latest.o

In [21]:
def process_buffer(buf):
    tnode = ET.fromstring(buf)
    return tnode.attrib['k']+', '+tnode.attrib['v']+'\n'

In [25]:
with open('pairs.txt', 'a') as the_file:
    for osm in osmfiles:
        inputbuffer = ''
        with open(osm,'r') as inputfile:
            print('parsing {}'.format(osm))
            for line in inputfile:
                if '<tag' in line:
                    inputbuffer = line
                    pair = process_buffer(inputbuffer)
                    del inputbuffer #probably redundant...
                    the_file.write(pair)

parsing ./states/missouri-latest.osm
parsing ./states/louisiana-latest.osm
parsing ./states/new-york-latest.osm
parsing ./states/new-hampshire-latest.osm
parsing ./states/west-virginia-latest.osm
parsing ./states/delaware-latest.osm
parsing ./states/new-jersey-latest.osm
parsing ./states/wyoming-latest.osm
parsing ./states/wisconsin-latest.osm
parsing ./states/washington-latest.osm
parsing ./states/north-carolina-latest.osm
parsing ./states/arizona-latest.osm
parsing ./states/nevada-latest.osm
parsing ./states/virginia-latest.osm
parsing ./states/oregon-latest.osm
parsing ./states/kentucky-latest.osm
parsing ./states/connecticut-latest.osm
parsing ./states/utah-latest.osm
parsing ./states/hawaii-latest.osm
parsing ./states/maryland-latest.osm
parsing ./states/ohio-latest.osm
parsing ./states/rhode-island-latest.osm
parsing ./states/iowa-latest.osm
parsing ./states/south-dakota-latest.osm
parsing ./states/maine-latest.osm
parsing ./states/nebraska-latest.osm
parsing ./states/illinois-la

In [2]:
sc = pyspark.SparkContext(appName="osm_tags_count_US")

In [3]:
pairs = sc.textFile('pairs.txt')
counts = pairs.flatMap(lambda whole: whole.split("\n")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .filter(lambda x: x[1] > 10000) \
             .sortBy(lambda x: x[1], ascending=False)
counts.repartition(1).saveAsTextFile('kv_count')

In [9]:
pairs = sc.textFile('pairs.txt')
counts = pairs.flatMap(lambda whole: whole.split("\n")) \
             .map(lambda word: (word.split(',')[0], 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .sortBy(lambda x: x[1], ascending=False)
counts.repartition(1).saveAsTextFile('key_count')